# Getting Started with ∆F Strings for Formatting

### Description
__∆F__ is a utility for Dyalog APL that performs a function similar to string formatters in other languages, such as Python's f-strings, but with an _APL_ flair. 

### Outline
* Setup
* ∆F Fields
  * Text Fields
      * Escapes
  * Code Fields
  * Debug Mode
  * DQ Strings 
      * Escapes
  * Space Fields
  * Pseudo-Builtin \$ for ⎕FMT formatting
  * Pseudo-builtin $$ for a boxed display
  * ∆F arguments: ⍹0, ⍹, etc.
  * Justification and Centering with \$
* Miscellaneous Options
  * Code Field namespace ⍺
  * Assertions with ∆F

### Setup
<br>Let's get started!

Let's be sure the file __∆Format.dyalog__ is accessible and loaded. When fixed, it creates a single function `∆F`.

In [1]:
⍝ We can start the Jupyter _ipynb_ file up in the same directory as ∆Format.dyalog. 
⎕SH 'pwd'
⎕←2 ⎕FIX 'file://∆Format.dyalog'

/Users/petermsiegel/MyDyalogLibrary/pmsLibrary/src

∆F

### ∆F Fields
####  Text Fields: `'A simple string'`
The simplest possible format string-- we'll call it an __∆F__ string-- consists of a simple __Text__ field.

In [2]:
∆F 'This is a simple string.'

This is a simple string.

##### Text Fields: Newlines and Escapes
__Text__ fields can consist of one or more lines, each separated by the special newline escape sequence `\⋄`. Using __Text__ fields this way is just one way to create a list of items or a multiline paragraph. 

In [3]:
∆F 'This\⋄is a\⋄multiline string.'

This 
is a 
multiline string.

You can insert most any Unicode character into a __Text__ field. Only four characters (with special meaning _described below_) require special treatment: 

`{`, `}`, `⋄`, and `\`.  
   
 A diamond `⋄` _not_ preceded by a backslash `\` has no special meaning; only the escaped sequence `\⋄` denotes a newline.  You'll see below that a bare `{` begins a __Code__ field, which is terminated by a balancing bare `}`. Finally, literal backslashes only need to be escaped themselves (i.e. as `\\`) when right before a `⋄`, `{`, `}`, or another backslash `\`. In other cases, such as `+\⍳` no extra backslash is required.

In [26]:
∆F '"We can sum the numbers ⍵ via +\⍵"'

"We can sum the numbers ⍵ via +\⍵".

#### Code Fields: Simple Variables `'{lastName}, {firstName}'`
Let's create a more useful example of ∆F strings using the following three variables. 

In [27]:
what←'This'
type←'simple'
thing←'string'

Within separate sets of curly braces `{..}`, which delimit a __Code__ field, we include the three variable names: `what`, `type`, and `thing`. We'll say more about __Code__ fields in a moment.

In [5]:
∆F '{what} is a {type} {thing}.'

This is a simple string.

#### Knowing Your Fields
This ∆F string consists of six fields: 
   1. a __Code__ field `{what}`, which returns the value of the variable `what`; 
   2. a __Text__ field `" is a "`;
   3. another __Code__ field `{type}`, returning the value of the variable `type`;
   4. a short __Text__ field `" "`; 
   5. a __Code__ field `{thing}`, referencing `thing`; and finally,
   6. a final __Text__ field `"."`.

##### Debug Mode: `'debug'`
We can show each of the fields more graphically using the _debug_ option (abbreviated _d_), which places each field in a separate display box and marks each space in each field by a middle dot `·`.

In [6]:
'd' ∆F '{what} is a {type} {thing}.'

┌→───┐┌→─────┐┌→─────┐┌→┐┌→─────┐┌→┐
↓This│↓·is·a·│↓simple│↓·│↓string│↓.│
└────┘└──────┘└──────┘└─┘└──────┘└─┘

#### Code Fields Are DFNS: `'{⍪⍳3} {↑"Name" "Addr" "Phone"}'`
As shown above, in addition to __Text__ fields, we can create executable __Code__ fields, using braces {...}. A __Code__ field with a bare variable names is the simplest type of __Code__ field.

__Code__ fields can be generalized as dfns* evaluated in the active (caller's) namespace. While each __Code__ field is executed via ordinary APL rules (statements left-to-right and right-to-left within statements), __Code__ fields within a `∆F` format string are themselves executed left-to-right.
Each __Code__ field _must_ return a value. 

                      _______________________________________
                      * Code fields may include comments, which each start with a lamp `⍝` 
                        and contain no braces or ⋄:  { ?0  ⍝ A random number ⍵: 0<⍵<1 }

Let's look at more complex examples. First, what if a variable itself is more than a simple one-line text string?

In [7]:
list←⍪⍳3
what←↑'This' 'That' 'The other thing'  
type←↑'simple' 'hard' 'confusing'
thing←↑'string' 'matrix' 'thingamabob'
∆F '{list} {what} is a {type} {thing}'

0 This is a simple string 
1 That hard matrix 
2 The other thing confusing thingamabob

Here, `list` is a column vector of integers, and `what`, `type` and `thing` are character matrices. Any object that can be formatted via Dyalog `⎕FMT` can be the result of a __Code__ field.

Now for a more complex example. You can place arbitrary APL code within the braces `{...}` of a __Code__ field. 

In the example below, we'll remove the ↑ prefix from the values of each of these three variables. Notice how we insert a period after each word of the variable `thing` and create a quoted string using double quotes: `{ ↑thing,¨"." }` Such a string is called a __DQ String__.

In [29]:
what←'This' 'That' 'The other thing'
type←'simple' 'hard' 'confusing'
thing←'string' 'matrix' 'thingamabob'
∆F '{ ⍪⍳≢what } { ↑what } is a { ↑type } { ↑thing,¨"." }'    ⍝ See below regarding "."

0 This is a simple string. 
1 That hard matrix. 
2 The other thing confusing thingamabob.

##### DQ Strings in Code Fields: Use `"These"` _not_ `'These'`
Within __Code__ fields, strings *require* double quotes (`"`). These __DQ strings__ `"like this one"` are used wherever single-quoted strings `'like this'` would be used in standard APL; single-quoted expressions are ***not*** used. Single quotes may appear, most usually as literal characters, rather than to create strings.

##### DQ Strings: Escapes  `\⋄` 
__DQ Strings__ support the escaped sequences `\⋄` and `\\`. To include an actual double quote within a __DQ String__, double the doublequote (`""`), just as one would do for single quotes in standard APL strings. Single quotes are doubled on entry as required by APL when entering the ∆F format string:

In [9]:
∆F '{"This\⋄is a\⋄""DQ"" field, isn''t it?"}'

This 
is a 
"DQ" field, isn't it?

#### Space Fields: `{  }`
The third and last field type is a <b>Space</b> field, which looks just like a <b>Code</b> field, except that it contains only zero or more spaces between the braces { }*. These spaces will form a separate field. 

                       _______________________________________
                       * Space fields may include comments, which each start with a lamp `⍝` 
                         and contain no braces or ⋄:   {   ⍝ 3 spaces} 

In [10]:
∆F 'This is{ }a test.'
'd' ∆F 'This is{ }a test.'

This is a test.

┌→──────┐┌→┐┌→──────┐
↓This·is│↓·│↓a·test.│
└───────┘└─┘└───────┘

But why bother with space fields? 
- They are useful when separating out multiline string or code fields; even a zero-width space field can separate two __Text__ fields; and 
- They ensure the expected amount of spacing when preceded or followed by text fields with lines of varying length.

Here's an example of two multiline __Text__ field separated by a __Space__ field with a single space: { }.

In [30]:
∆F 'This\⋄is a\⋄multiline\⋄field!{ ⍝ Space}{"This\⋄is\⋄as well!"}'

This This 
is a is 
multiline as well!
field!

In this next example, we use a zero-width __Space__ field simply to allow us to create two independent __Text__ fields:

In [12]:
∆F '1. \⋄2.\⋄3.{}Jane\⋄John\⋄Nancy'   ⍝ Or equivalent:  '1.\⋄2.\⋄3.{ }Jane\⋄John\⋄Nancy'

1. Jane 
2. John 
3. Nancy

#### Pseudo-builtin `$`
Here's how to do this more elegantly using the pseudo-builtin `$`, which is a nice way to use the Dyalog APL formatting utility `⎕FMT`. 

In [33]:
∆F '{"I1,⊂. ⊃" $ 1+⍳3}Jane\⋄John\⋄Nancy'

1. Jane 
2. John 
3. Nancy

Now, let's move on to a few more examples.

In [34]:
∆F 'Multiples of pi: {"I1,⊂×Π =⊃" $ 1+⍳4} {"F10.7" $ ○1 2 3 4}'

Multiples of pi: 1×Π = 3.1415927
 2×Π = 6.2831853
 3×Π = 9.4247780
 4×Π = 12.5663706

Again, using the *debug* option, we can see exactly what fields are set up.

In [14]:
'd' ∆F 'Multiples of pi: {"I1,⊂×Π =⊃" $ 1+⍳4} {"F10.7" $ ○1 2 3 4}'

┌→────────────────┐┌→────┐┌→┐┌→─────────┐
↓Multiples·of·pi:·│↓1×Π·=│↓·│↓·3.1415927│
└─────────────────┘│2×Π·=│└─┘│·6.2831853│
 │3×Π·=│ │·9.4247780│
 │4×Π·=│ │12.5663706│
 └─────┘ └──────────┘

#### Pseudo-builtin Function $$ for a boxed display
If we want a __Code__ field to be boxed in the regular output, we can use the pseudo-builtin display function __\$\$__. Using __\$\$__, no middle dots (·) appear, unless you create them yourself!

In [15]:
∆F 'Multiples of pi: {$$ "I1,⊂×Π =⊃" $ 1+⍳4} {$$ "F10.7" $ ○1 2 3 4}'

Multiples of pi: ┌→────┐ ┌→─────────┐
 ↓1×Π =│ ↓ 3.1415927│
 │2×Π =│ │ 6.2831853│
 │3×Π =│ │ 9.4247780│
 │4×Π =│ │12.5663706│
 └─────┘ └──────────┘

#### ∆F arguments with `⍹0` ... `⍹99` and `⍹`   (or `⍵0` ... `⍵99` and `⍵_`)
∆F supports the use of arguments to __∆F__, including the `∆F` format string itself.  The format string is designated `⍹0`, and each subsequent argument is `⍹1`, `⍹2`, etc. These designations can be used in place of `((0+⎕IO)⊃⍵)`, et cetera, within __Code__ fields:

In [16]:
∆F '{⍹1} multiples of pi: {"I1,⊂×Π =⊃" $ 1+⍳⍹1 } {"F10.7" $ ○ 1+⍳⍹1}'  3  
∆F '{⍹1} multiples of pi: {"I1,⊂×Π =⊃" $ 1+⍳⍹1 } {"F10.7" $ ○ 1+⍳⍹1}'  2  

3 multiples of pi: 1×Π = 3.1415927
 2×Π = 6.2831853
 3×Π = 9.4247780

2 multiples of pi: 1×Π = 3.1415927
 2×Π = 6.2831853

The symbol ∆F `⍹` alone will select the **next** argument in sequence (one past the **current** argument, which  is the last one selected directly, e.g. `⍹5`, via `⍹`, or `⍹0` if the first use). This makes it easy to format a set of items: 

In [17]:
∆F 'Rate: {⍹ $ ⍹}; cur. value: {⍹ $ ⍹}' 'F5.2,⊂%⊃' (2.200 3.834 5.996) '⊂£⊃,F7.2' (1000.23, 2250.19 2500.868)

Rate: 2.20%; cur. value: £1000.23
 3.83% £2250.19
 6.00% £2500.87

Note: You may enter `⍵0`, `⍵1`, etc. as equivalents to `⍹0`, `⍹1`, etc. and `⍵_` for `⍹` alone. Note also that `⍹0` can never be selected via the lone ⍹, since the *last* index specified is never less than 0, so the *next* is never less than 1.

#### Justification and Centering with  Pseudo-function `$`
The pseudo-function `$` has been extended with 3 special codes for left-justified (Lnn), centered (Cnn), and right-justified (Rnn) output within a **Code** field. This is valid for both numeric and text data.  Only one special code may be used in each `$` call (but you may call $ itself more than once) and that code must be the *first* or *only* code specified. If other (usually numerically-oriented) codes  follow, a comma must intervene (following the style of dyadic `⎕FMT`).

Here, we *left-, center-*, and *right-*justify three items in the ∆F arguments: ⍵1, ⍵2, ⍵3:

In [18]:
    ∆F 'Surname <{"L10" $ ⍵_}>  First <{"C10" $ ⍵_}>. Salary <{"R10,⊂$⊃,F7.2" $ ⍵_}>' 'Smith' 'Mary' 2534.23

Surname <Smith > First < Mary >. Salary < $2534.23>

In [19]:
'd' ∆F 'Surname <{"L10" $ ⍵_}>  First <{"C10" $ ⍵_}>. Salary <{"R10,⊂$⊃,F7.2" $ ⍵_}>' 'Smith' 'Mary' 2534.23

┌→────────┐┌→─────────┐┌→─────────┐┌→─────────┐┌→──────────┐┌→─────────┐┌→┐
↓Surname·<│↓Smith·····│↓>··First·<│↓···Mary···│↓>.·Salary·<│↓··$2534.23│↓>│
└─────────┘└──────────┘└──────────┘└──────────┘└───────────┘└──────────┘└─┘

### Miscellaneous Options
#### Code field namespace ⍺
For **Code** fields, ∆F passes a namespace as the left argument (⍺). That namespace contains all the support functions and variables for ∆F. Names beginning with an underscore `_` (e.g. `_`, `__`, `_myVar`, ...) are reserved for the user's use. One potential use is setting state that is maintained across all **Code** fields during the execution of ∆F, without cluttering the calling environment:

In [20]:
∆F '{⍺._PITimes←(○1)∘× ⋄ ⍺._PITimes 1} {⍺._PITimes 2}' 

3.141592654 6.283185307

In [21]:
⎕FR←645
∆F '{⍺._sav←⎕FR  ⋄ ⎕FR∘←1287  ⋄ (⎕FR∘←⍺._sav)⊢$ ○1 }'
'⎕FR still 645? ',(⎕FR=645)⊃'No' 'Yes'

3.141592654

⎕FR still 645? Yes

#### ∆F for assertions.
Normally, ∆F returns the formatted text as a single formatted matrix (rank 2).
<br>If the left argument (⍺) to ∆F is a homogeneous numeric array, it is viewed as an assertion.
  * If the assertion contains _no numeric zeroes_, it is __true__. It **prints** the formatted text, returning a shy _1_. (It does _not_ return the formatted text, as in _format_ mode.)
  * If the assertion contains one or more zeroes, it is __false__. It does nothing, returning a shy _0_.

In [22]:
⍝ Here, var is in range, so no ∆F string message is produced. 0 is returned.
var←100
⎕←(var<100) ∆F 'Warning! Variable "var" is out of range: var={var}'


0

In [23]:
⍝ Now, var is out of range (the assertion is true), so a ∆F string message is printed. 1 is returned.
var-←1
⎕←(var<100) ∆F 'Warning! Variable "var" is out of range: var={var}'

Warning! Variable "var" is out of range: var=99
1